In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pandas as pd
from datetime import date

import bt

import matplotlib.pyplot as plt
from ffn import *
from importlib import reload
import scipy.stats as stats
from scipy.stats import mstats
from scipy.stats import zscore
   

In [1]:
# here we read file containing 220+ security which will be used for momentum calcuation

In [2]:
# read the file containing list of NSE 200 
df_list = pd.read_csv("NSE_200_list.csv")

In [3]:
df_list.head()

,Company Name,BB Symbol,ISIN,Sector,Data
0,ABB India Ltd.,ABB IN Equity,INE117A01022,Industrials,Yes
1,Aditya Birla Capital Ltd.,ABCAP IN Equity,INE674K01013,Financials,Yes
2,Aditya Birla Real Estate Limited.,ABREL IN Equity,INE055A01016,Real Estate,No
3,Aditya Birla Fashion and Retail Ltd.,ABFRL IN Equity,INE647O01011,Consumer Discretionary,Yes
4,ACC Ltd.,ACC IN Equity,INE012A01025,Materials,Yes


In [4]:
df_list[df_list['Data']=='No']

,Company Name,BB Symbol,ISIN,Sector,Data
2,Aditya Birla Real Estate Limited.,ABREL IN Equity,INE055A01016,Real Estate,No
32,Bharti Hexacom Ltd.,BHARTIHE IN Equity,INE054A01027,Telecommunications,No
63,Delhivery Ltd.,DELHIVER IN Equity,INE148O01028,Industrials,No
118,Indian Renewable Energy Development Agency Ltd.,IREDA IN Equity,INE202G01020,Financials,No
119,Indian Railway Finance Corporation Ltd.,IRFC IN Equity,INE053F01010,Financials,No
121,JIO Financial Services Ltd.,JIOFIN IN Equity,INE758E01017,Financials,No
126,JSW Infrastructure Ltd.,JSWINFRA IN Equity,INE391O01016,Industrials,No
128,Kalyan Jewellers India Ltd.,KALYANKJ IN Equity,INE303R01014,Consumer Discretionary,No
133,Life Insurance Corporation of India.,LICI IN Equity,INE0J1Y01017,Financials,No
134,Macrotech Developers Ltd.,LODHA IN Equity,INE670K01029,Real Estate,No


In [5]:
#df_list = df_list[df_list['Data']=='Yes']

In [6]:
df_list.describe()

,Company Name,BB Symbol,ISIN,Sector,Data
count,236,236,236,236,236
unique,236,236,236,12,2
top,ABB India Ltd.,ABB IN Equity,INE117A01022,Financials,Yes
freq,1,1,1,55,219


In [2]:
#  we load the file containing daily prices

In [7]:
df_data = pd.read_csv("Price_17_24.csv")

In [8]:
df_data.head()

,Company Name,CD_NSE Symbol,CD_Bloomberg Code,NDP_Date,NDP_Close,BB_Symbol
0,Aarti Industries Ltd.,AARTIIND,ARTO IN,02-01-2017,173.81,ARTO IN Equity
1,Aarti Industries Ltd.,AARTIIND,ARTO IN,03-01-2017,175.85,ARTO IN Equity
2,Aarti Industries Ltd.,AARTIIND,ARTO IN,04-01-2017,174.50,ARTO IN Equity
3,Aarti Industries Ltd.,AARTIIND,ARTO IN,05-01-2017,175.34,ARTO IN Equity
4,Aarti Industries Ltd.,AARTIIND,ARTO IN,06-01-2017,174.94,ARTO IN Equity


In [9]:
df_data.tail()

,Company Name,CD_NSE Symbol,CD_Bloomberg Code,NDP_Date,NDP_Close,BB_Symbol
433444,Life Insurance Corporation of India,LICI,LICI IN,11-11-2024,918.45,LICI IN Equity
433445,Life Insurance Corporation of India,LICI,LICI IN,12-11-2024,921.15,LICI IN Equity
433446,Life Insurance Corporation of India,LICI,LICI IN,13-11-2024,897.90,LICI IN Equity
433447,Life Insurance Corporation of India,LICI,LICI IN,14-11-2024,908.65,LICI IN Equity
433448,Life Insurance Corporation of India,LICI,LICI IN,18-11-2024,901.75,LICI IN Equity


In [10]:
df_data['CD_Bloomberg Code'].nunique()

235

In [11]:
df_data = df_data.drop(columns=['CD_NSE Symbol','CD_Bloomberg Code'])

In [12]:
df_data.head()

,Company Name,NDP_Date,NDP_Close,BB_Symbol
0,Aarti Industries Ltd.,02-01-2017,173.81,ARTO IN Equity
1,Aarti Industries Ltd.,03-01-2017,175.85,ARTO IN Equity
2,Aarti Industries Ltd.,04-01-2017,174.50,ARTO IN Equity
3,Aarti Industries Ltd.,05-01-2017,175.34,ARTO IN Equity
4,Aarti Industries Ltd.,06-01-2017,174.94,ARTO IN Equity


In [13]:
df_data = df_data.rename(columns={'NDP_Date': 'date', 'BB_Symbol':'BB Symbol'})

In [14]:
df_data.head()

,Company Name,date,NDP_Close,BB Symbol
0,Aarti Industries Ltd.,02-01-2017,173.81,ARTO IN Equity
1,Aarti Industries Ltd.,03-01-2017,175.85,ARTO IN Equity
2,Aarti Industries Ltd.,04-01-2017,174.50,ARTO IN Equity
3,Aarti Industries Ltd.,05-01-2017,175.34,ARTO IN Equity
4,Aarti Industries Ltd.,06-01-2017,174.94,ARTO IN Equity


In [15]:
df_data = pd.merge(df_data, df_list[['BB Symbol','ISIN', 'Sector']], on='BB Symbol', how='left')

In [16]:
df_data.head()

,Company Name,date,NDP_Close,BB Symbol,ISIN,Sector
0,Aarti Industries Ltd.,02-01-2017,173.81,ARTO IN Equity,INE769A01020,Materials
1,Aarti Industries Ltd.,03-01-2017,175.85,ARTO IN Equity,INE769A01020,Materials
2,Aarti Industries Ltd.,04-01-2017,174.50,ARTO IN Equity,INE769A01020,Materials
3,Aarti Industries Ltd.,05-01-2017,175.34,ARTO IN Equity,INE769A01020,Materials
4,Aarti Industries Ltd.,06-01-2017,174.94,ARTO IN Equity,INE769A01020,Materials


In [17]:
df_data.shape

(433449, 6)

In [18]:
df_data = df_data.dropna(subset=['BB Symbol'])

In [19]:
df_data.isna().sum()

Company Name    0
date            0
NDP_Close       0
BB Symbol       0
ISIN            0
Sector          0
dtype: int64

In [20]:
df_data['BB Symbol'].nunique()

235

In [21]:
#df_data = df_data.drop(columns=['CD_ISIN No'])

In [22]:
df_data.head()

,Company Name,date,NDP_Close,BB Symbol,ISIN,Sector
0,Aarti Industries Ltd.,02-01-2017,173.81,ARTO IN Equity,INE769A01020,Materials
1,Aarti Industries Ltd.,03-01-2017,175.85,ARTO IN Equity,INE769A01020,Materials
2,Aarti Industries Ltd.,04-01-2017,174.50,ARTO IN Equity,INE769A01020,Materials
3,Aarti Industries Ltd.,05-01-2017,175.34,ARTO IN Equity,INE769A01020,Materials
4,Aarti Industries Ltd.,06-01-2017,174.94,ARTO IN Equity,INE769A01020,Materials


In [23]:
df_data = df_data.rename(columns={'NDP_Close': 'Price'})

In [24]:
df_data.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector
0,Aarti Industries Ltd.,02-01-2017,173.81,ARTO IN Equity,INE769A01020,Materials
1,Aarti Industries Ltd.,03-01-2017,175.85,ARTO IN Equity,INE769A01020,Materials
2,Aarti Industries Ltd.,04-01-2017,174.50,ARTO IN Equity,INE769A01020,Materials
3,Aarti Industries Ltd.,05-01-2017,175.34,ARTO IN Equity,INE769A01020,Materials
4,Aarti Industries Ltd.,06-01-2017,174.94,ARTO IN Equity,INE769A01020,Materials


In [25]:
df_data['date'] = pd.to_datetime(df_data['date'], dayfirst=True)
df_data = df_data.sort_values(by='date')

In [26]:
df_data.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector
0,Aarti Industries Ltd.,2017-01-02,173.81,ARTO IN Equity,INE769A01020,Materials
36124,Apollo Hospitals Enterprise Ltd.,2017-01-02,1189.20,APHS IN Equity,INE437A01024,Health Care
317750,Prestige Estates Projects Ltd.,2017-01-02,169.00,PEPL IN Equity,INE811K01011,Real Estate
319703,Punjab National Bank,2017-01-02,115.60,PNB IN Equity,INE160A01022,Financials
323045,Rajesh Exports Ltd.,2017-01-02,466.20,RJEX IN Equity,INE343B01030,Consumer Discretionary


In [27]:
df_data.dtypes

Company Name            object
date            datetime64[ns]
Price                  float64
BB Symbol               object
ISIN                    object
Sector                  object
dtype: object

In [28]:
df_data.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector
0,Aarti Industries Ltd.,2017-01-02,173.81,ARTO IN Equity,INE769A01020,Materials
36124,Apollo Hospitals Enterprise Ltd.,2017-01-02,1189.20,APHS IN Equity,INE437A01024,Health Care
317750,Prestige Estates Projects Ltd.,2017-01-02,169.00,PEPL IN Equity,INE811K01011,Real Estate
319703,Punjab National Bank,2017-01-02,115.60,PNB IN Equity,INE160A01022,Financials
323045,Rajesh Exports Ltd.,2017-01-02,466.20,RJEX IN Equity,INE343B01030,Consumer Discretionary


In [29]:
df_mom = df_data.copy()

In [30]:
df_mom.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector
0,Aarti Industries Ltd.,2017-01-02,173.81,ARTO IN Equity,INE769A01020,Materials
36124,Apollo Hospitals Enterprise Ltd.,2017-01-02,1189.20,APHS IN Equity,INE437A01024,Health Care
317750,Prestige Estates Projects Ltd.,2017-01-02,169.00,PEPL IN Equity,INE811K01011,Real Estate
319703,Punjab National Bank,2017-01-02,115.60,PNB IN Equity,INE160A01022,Financials
323045,Rajesh Exports Ltd.,2017-01-02,466.20,RJEX IN Equity,INE343B01030,Consumer Discretionary


In [31]:
df_mom['returns'] = df_mom.sort_values(by='date').groupby('BB Symbol')['Price'].pct_change()*100
df_mom = df_mom.dropna(subset=['returns'])

In [32]:
df_mom.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns
381969,Tata Steel Ltd.,2017-01-03,38.42,TATA IN Equity,INE081A01020,Materials,-0.851613
324999,REC Ltd.,2017-01-03,94.80,RECL IN Equity,INE020B01018,Financials,1.488063
300741,Petronet LNG Ltd.,2017-01-03,190.38,PLNG IN Equity,INE347G01014,Energy,2.669471
109672,Container Corporation Of India Ltd.,2017-01-03,376.58,CCRI IN Equity,INE111A01025,Industrials,3.133045
419351,Vedanta Ltd.,2017-01-03,219.65,VEDL IN Equity,INE205A01025,Materials,-0.588368


In [33]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000


In [34]:
### Momentum calcualtions

In [35]:
# 6 month momentum

In [36]:
# convert return to log value
df_mom['log_return'] = np.log((df_mom['returns']/100)+1)

In [37]:


df_mom['raw_mom'] = df_mom.groupby('BB Symbol')['log_return'].transform(
    lambda x: x.rolling(126).apply(np.sum, raw=True)
)


In [38]:
df_mom['momentum_6'] = df_mom.sort_values(by='date').groupby('date')['raw_mom'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

In [39]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,log_return,raw_mom,momentum_6
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,-0.015693,0.003415,0.050862
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,-0.035412,0.092343,0.535353
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.000598,-0.101583,-0.521178
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,-0.006330,0.102953,0.593157
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,0.000000,-0.173270,-0.911739


In [40]:
df_mom.shape

(433214, 10)

In [41]:
df_mom['rolling_std_dev'] = df_mom.sort_values(by='date').groupby('BB Symbol')['log_return'].transform(lambda x: (x.rolling(126).std()))

In [42]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,log_return,raw_mom,momentum_6,rolling_std_dev
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,-0.015693,0.003415,0.050862,0.022834
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,-0.035412,0.092343,0.535353,0.021612
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.000598,-0.101583,-0.521178,0.026579
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,-0.006330,0.102953,0.593157,0.032412
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,0.000000,-0.173270,-0.911739,0.009919


In [43]:
df_mom['raw_mom_RA'] = df_mom['raw_mom']/df_mom['rolling_std_dev']

In [44]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,log_return,raw_mom,momentum_6,rolling_std_dev,raw_mom_RA
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,-0.015693,0.003415,0.050862,0.022834,0.149540
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,-0.035412,0.092343,0.535353,0.021612,4.272732
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.000598,-0.101583,-0.521178,0.026579,-3.822005
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,-0.006330,0.102953,0.593157,0.032412,3.176331
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,0.000000,-0.173270,-0.911739,0.009919,-17.468355


In [45]:
df_mom['momentum_RA_6'] = df_mom.sort_values(by='date').groupby('date')['raw_mom_RA'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

In [46]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,log_return,raw_mom,momentum_6,rolling_std_dev,raw_mom_RA,momentum_RA_6
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,-0.015693,0.003415,0.050862,0.022834,0.149540,0.026558
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,-0.035412,0.092343,0.535353,0.021612,4.272732,0.498512
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.000598,-0.101583,-0.521178,0.026579,-3.822005,-0.428038
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,-0.006330,0.102953,0.593157,0.032412,3.176331,0.373015
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,0.000000,-0.173270,-0.911739,0.009919,-17.468355,-1.990044


In [47]:
df_mom = df_mom.drop(['log_return','raw_mom','rolling_std_dev', 'raw_mom_RA'],axis=1)

In [48]:
df_mom.shape

(433214, 9)

In [49]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,0.050862,0.026558
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,0.535353,0.498512
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.521178,-0.428038
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,0.593157,0.373015
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,-0.911739,-1.990044


In [50]:
## 9 month mometum

In [51]:
# convert return to log value
df_mom['log_return'] = np.log((df_mom['returns']/100)+1)

In [52]:

df_mom['raw_mom'] = df_mom.groupby('BB Symbol')['log_return'].transform(
    lambda x: x.rolling(189).apply(np.sum, raw=True)
)

df_mom['momentum_9'] = df_mom.sort_values(by='date').groupby('date')['raw_mom'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

df_mom['rolling_std_dev'] = df_mom.sort_values(by='date').groupby('BB Symbol')['log_return'].transform(lambda x: (x.rolling(189).std()))

df_mom['raw_mom_RA'] = df_mom['raw_mom']/df_mom['rolling_std_dev']

df_mom['momentum_RA_9'] = df_mom.sort_values(by='date').groupby('date')['raw_mom_RA'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

df_mom = df_mom.drop(['log_return','raw_mom','rolling_std_dev', 'raw_mom_RA'],axis=1)

In [53]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,momentum_RA_9
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,0.050862,0.026558,0.216085,0.229779
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,0.535353,0.498512,1.261536,1.279849
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.521178,-0.428038,-0.506876,-0.504415
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,0.593157,0.373015,1.433147,0.880160
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,-0.911739,-1.990044,-0.754328,-1.019428


In [54]:
## 12 months momentum

In [55]:
df_mom['log_return'] = np.log((df_mom['returns']/100)+1)

df_mom['raw_mom'] = df_mom.groupby('BB Symbol')['log_return'].transform(
    lambda x: x.rolling(252).apply(np.sum, raw=True)
)

df_mom['momentum_12'] = df_mom.sort_values(by='date').groupby('date')['raw_mom'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

df_mom['rolling_std_dev'] = df_mom.sort_values(by='date').groupby('BB Symbol')['log_return'].transform(lambda x: (x.rolling(252).std()))

df_mom['raw_mom_RA'] = df_mom['raw_mom']/df_mom['rolling_std_dev']

df_mom['momentum_RA_12'] = df_mom.sort_values(by='date').groupby('date')['raw_mom_RA'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

df_mom = df_mom.drop(['log_return','raw_mom','rolling_std_dev', 'raw_mom_RA'],axis=1)

In [56]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,momentum_RA_9,momentum_12,momentum_RA_12
284694,NTPC Ltd.,2024-11-18,366.70,NTPC IN Equity,INE733E01010,Utilities,-1.557047,0.050862,0.026558,0.216085,0.229779,0.569888,0.702334
286647,Oberoi Realty Ltd.,2024-11-18,1911.40,OBER IN Equity,INE093I01010,Real Estate,-3.479271,0.535353,0.498512,1.261536,1.279849,0.603717,0.637882
288600,Oil & Natural Gas Corporation Ltd.,2024-11-18,250.65,ONGC IN Equity,INE213A01029,Energy,-0.059809,-0.521178,-0.428038,-0.506876,-0.504415,-0.013037,-0.065523
290553,Oil India Ltd.,2024-11-18,472.45,OINL IN Equity,INE274J01014,Energy,-0.630981,0.593157,0.373015,1.433147,0.880160,2.095964,1.276510
433448,Life Insurance Corporation of India,2024-11-18,901.75,LICI IN Equity,INE0J1Y01017,Financials,0.000000,-0.911739,-1.990044,-0.754328,-1.019428,-1.141315,-1.248321


In [57]:
### MIF (based on 12 months)

In [58]:
df_mom = df_mom.sort_values(by=['BB Symbol', 'date'])

In [59]:
for months in [1]:
    
    # We assume a month has roughly 21 trading days.
    trading_days = months * 21
    df_mom[f'1st_order_delta_{months}_month'] = df_mom.groupby('BB Symbol')['momentum_12'].diff(periods=trading_days)
    df_mom[f'2nd_order_delta_{months}_month'] = df_mom.groupby('BB Symbol')[f'1st_order_delta_{months}_month'].diff(periods=trading_days)

In [60]:
df_mom['sum_delta'] = df_mom['1st_order_delta_1_month']*0.5 +df_mom['2nd_order_delta_1_month']*0.5

In [61]:
df_mom['MIF'] = df_mom.sort_values(by='date').groupby('date')['sum_delta'].transform(lambda x: zscore(x,ddof=1, nan_policy='omit'))

In [62]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,momentum_RA_9,momentum_12,momentum_RA_12,1st_order_delta_1_month,2nd_order_delta_1_month,sum_delta,MIF
432354,Zydus Lifesciences Ltd.,2024-11-11,967.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,-0.267945,-0.472658,-0.507847,0.449729,0.598552,0.661150,1.102713,-0.043278,-0.089577,-0.066427,-0.152311
432355,Zydus Lifesciences Ltd.,2024-11-12,949.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,-1.859985,-0.408296,-0.464076,0.363711,0.479891,0.507083,0.905266,-0.151308,-0.181170,-0.166239,-0.375194
432356,Zydus Lifesciences Ltd.,2024-11-13,963.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,1.474072,-0.259404,-0.320174,0.480581,0.618899,0.712318,1.154396,0.027719,0.002294,0.015007,-0.004314
432357,Zydus Lifesciences Ltd.,2024-11-14,957.40,ZYDUSLIF IN Equity,INE010B01027,Health Care,-0.658885,-0.324313,-0.400657,0.095832,0.202667,0.647210,1.066138,0.060033,0.167392,0.113712,0.213833
432358,Zydus Lifesciences Ltd.,2024-11-18,944.70,ZYDUSLIF IN Equity,INE010B01027,Health Care,-1.326509,-0.550747,-0.693321,0.064195,0.157638,0.607921,1.003577,-0.084088,-0.066260,-0.075174,-0.215221


In [63]:
df_mom = df_mom.drop(['1st_order_delta_1_month','2nd_order_delta_1_month','sum_delta'],axis=1)

In [64]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,momentum_RA_9,momentum_12,momentum_RA_12,MIF
432354,Zydus Lifesciences Ltd.,2024-11-11,967.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,-0.267945,-0.472658,-0.507847,0.449729,0.598552,0.661150,1.102713,-0.152311
432355,Zydus Lifesciences Ltd.,2024-11-12,949.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,-1.859985,-0.408296,-0.464076,0.363711,0.479891,0.507083,0.905266,-0.375194
432356,Zydus Lifesciences Ltd.,2024-11-13,963.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,1.474072,-0.259404,-0.320174,0.480581,0.618899,0.712318,1.154396,-0.004314
432357,Zydus Lifesciences Ltd.,2024-11-14,957.40,ZYDUSLIF IN Equity,INE010B01027,Health Care,-0.658885,-0.324313,-0.400657,0.095832,0.202667,0.647210,1.066138,0.213833
432358,Zydus Lifesciences Ltd.,2024-11-18,944.70,ZYDUSLIF IN Equity,INE010B01027,Health Care,-1.326509,-0.550747,-0.693321,0.064195,0.157638,0.607921,1.003577,-0.215221


In [65]:
df_mom = df_mom.dropna(subset=['momentum_6'])

In [66]:
df_mom.shape

(403839, 14)

In [67]:
#df_mom = df_mom.dropna(subset=['momentum_RA'])

In [68]:
df_mom.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,momentum_RA_9,momentum_12,momentum_RA_12,MIF
2079,ABB India Ltd.,2017-07-06,1451.30,ABB IN Equity,INE117A01022,Industrials,1.132365,0.694156,0.402368,NaN,NaN,NaN,NaN,NaN
2080,ABB India Ltd.,2017-07-07,1458.00,ABB IN Equity,INE117A01022,Industrials,0.461655,0.701052,0.415066,NaN,NaN,NaN,NaN,NaN
2081,ABB India Ltd.,2017-07-10,1451.45,ABB IN Equity,INE117A01022,Industrials,-0.449246,0.698314,0.391381,NaN,NaN,NaN,NaN,NaN
2082,ABB India Ltd.,2017-07-11,1464.60,ABB IN Equity,INE117A01022,Industrials,0.905991,0.855465,0.525743,NaN,NaN,NaN,NaN,NaN
2083,ABB India Ltd.,2017-07-12,1462.00,ABB IN Equity,INE117A01022,Industrials,-0.177523,0.745157,0.432910,NaN,NaN,NaN,NaN,NaN


In [69]:
#backup file 
backup_df = df_mom.copy()

In [70]:
df_mom = df_mom.dropna(subset=['momentum_12'])

In [71]:
df_mom.shape

(374229, 14)

In [72]:
df_mom.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,momentum_RA_9,momentum_12,momentum_RA_12,MIF
2205,ABB India Ltd.,2018-01-05,1435.1,ABB IN Equity,INE117A01022,Industrials,0.125584,-0.976181,-1.076045,-0.475453,-0.435330,-0.091721,-0.040910,NaN
2206,ABB India Ltd.,2018-01-08,1437.6,ABB IN Equity,INE117A01022,Industrials,0.174204,-1.027277,-1.169179,-0.444239,-0.381331,-0.127503,-0.077064,NaN
2207,ABB India Ltd.,2018-01-09,1419.3,ABB IN Equity,INE117A01022,Industrials,-1.272955,-1.035835,-1.202220,-0.779455,-0.817799,-0.136239,-0.091679,NaN
2208,ABB India Ltd.,2018-01-10,1415.4,ABB IN Equity,INE117A01022,Industrials,-0.274783,-1.111463,-1.332743,-0.853032,-0.926090,-0.089833,-0.043890,NaN
2209,ABB India Ltd.,2018-01-11,1399.6,ABB IN Equity,INE117A01022,Industrials,-1.116292,-1.128813,-1.388814,-0.997627,-1.142664,-0.176436,-0.140454,NaN


In [73]:
# Calculate the rank and decile for 'momentum'
df_mom['momentum_6_rank'] = df_mom.groupby('date')['momentum_6'].transform(lambda x: x.rank(ascending=False, method='dense'))
df_mom['momentum__6_decile'] = df_mom.groupby('date')['momentum_6'].transform(lambda x: pd.qcut(x.rank(ascending=False, method='first'), 10, labels=range(1, 11)))

# Calculate the rank and decile for 'momentum_RA'
df_mom['momentum_RA_6_rank'] = df_mom.groupby('date')['momentum_RA_6'].transform(lambda x: x.rank(ascending=False, method='dense'))
df_mom['momentum_RA_6_decile'] = df_mom.groupby('date')['momentum_RA_6'].transform(lambda x: pd.qcut(x.rank(ascending=False, method='first'), 10, labels=range(1, 11)))

In [74]:
# Calculate the rank and decile for 'momentum'
df_mom['momentum_9_rank'] = df_mom.groupby('date')['momentum_9'].transform(lambda x: x.rank(ascending=False, method='dense'))
df_mom['momentum__9_decile'] = df_mom.groupby('date')['momentum_9'].transform(lambda x: pd.qcut(x.rank(ascending=False, method='first'), 10, labels=range(1, 11)))

# Calculate the rank and decile for 'momentum_RA'
df_mom['momentum_RA_9_rank'] = df_mom.groupby('date')['momentum_RA_9'].transform(lambda x: x.rank(ascending=False, method='dense'))
df_mom['momentum_RA_9_decile'] = df_mom.groupby('date')['momentum_RA_9'].transform(lambda x: pd.qcut(x.rank(ascending=False, method='first'), 10, labels=range(1, 11)))

In [75]:
# Calculate the rank and decile for 'momentum'
df_mom['momentum_12_rank'] = df_mom.groupby('date')['momentum_12'].transform(lambda x: x.rank(ascending=False, method='dense'))
df_mom['momentum__12_decile'] = df_mom.groupby('date')['momentum_12'].transform(lambda x: pd.qcut(x.rank(ascending=False, method='first'), 10, labels=range(1, 11)))

# Calculate the rank and decile for 'momentum_RA'
df_mom['momentum_RA_12_rank'] = df_mom.groupby('date')['momentum_RA_12'].transform(lambda x: x.rank(ascending=False, method='dense'))
df_mom['momentum_RA_12_decile'] = df_mom.groupby('date')['momentum_RA_12'].transform(lambda x: pd.qcut(x.rank(ascending=False, method='first'), 10, labels=range(1, 11)))

In [76]:
# Calculate the rank and decile directly in df_mom
df_mom['MIF_rank'] = df_mom.groupby('date')['MIF'].transform(
    lambda x: x.rank(ascending=False, method='dense') if not x.isnull().all() else x
)

df_mom['MIF_decile'] = df_mom.groupby('date')['MIF'].transform(
    lambda x: pd.qcut(
        x.rank(ascending=False, method='first'),
        10,
        labels=range(1, 11)
    ) if not x.isnull().all() else x
)


In [77]:
df_mom.tail()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,...,momentum_9_rank,momentum__9_decile,momentum_RA_9_rank,momentum_RA_9_decile,momentum_12_rank,momentum__12_decile,momentum_RA_12_rank,momentum_RA_12_decile,MIF_rank,MIF_decile
432354,Zydus Lifesciences Ltd.,2024-11-11,967.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,-0.267945,-0.472658,-0.507847,0.449729,...,78.0,4,78.0,4,60.0,3,30.0,2,141.0,6.0
432355,Zydus Lifesciences Ltd.,2024-11-12,949.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,-1.859985,-0.408296,-0.464076,0.363711,...,88.0,4,85.0,4,71.0,3,40.0,2,161.0,7.0
432356,Zydus Lifesciences Ltd.,2024-11-13,963.75,ZYDUSLIF IN Equity,INE010B01027,Health Care,1.474072,-0.259404,-0.320174,0.480581,...,74.0,3,78.0,4,53.0,3,31.0,2,126.0,6.0
432357,Zydus Lifesciences Ltd.,2024-11-14,957.40,ZYDUSLIF IN Equity,INE010B01027,Health Care,-0.658885,-0.324313,-0.400657,0.095832,...,117.0,5,111.0,5,62.0,3,31.0,2,107.0,5.0
432358,Zydus Lifesciences Ltd.,2024-11-18,944.70,ZYDUSLIF IN Equity,INE010B01027,Health Care,-1.326509,-0.550747,-0.693321,0.064195,...,119.0,5,111.0,5,64.0,3,37.0,2,152.0,7.0


In [78]:
df_mom[df_mom['date']=='2021-07-01'].sort_values(by='momentum_6_rank')

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,...,momentum_9_rank,momentum__9_decile,momentum_RA_9_rank,momentum_RA_9_decile,momentum_12_rank,momentum__12_decile,momentum_RA_12_rank,momentum_RA_12_decile,MIF_rank,MIF_decile
313004,Poonawalla Fincorp Ltd.,2021-07-01,149.10,POONAWAL IN Equity,INE511C01022,Financials,1.428571,4.071448,2.515086,3.119988,...,3.0,1,12.0,1,4.0,1,18.0,1,215.0,10.0
10878,Adani Enterprises Ltd.,2021-07-01,1490.25,ADE IN Equity,INE423A01024,Industrials,-1.147557,3.488163,2.608220,3.700293,...,1.0,1,1.0,1,1.0,1,2.0,1,59.0,3.0
17872,Adani Total Gas Ltd.,2021-07-01,968.90,ATGL IN Equity,INE399L01023,Utilities,-4.995833,2.893711,1.371819,3.674432,...,2.0,1,11.0,1,3.0,1,11.0,1,214.0,10.0
389422,The Fertilisers And Chemicals Travancore Ltd.,2021-07-01,139.90,FCT IN Equity,INE188A01015,Materials,1.340094,2.811049,0.960759,2.154185,...,9.0,1,67.0,4,20.0,1,87.0,4,42.0,2.0
198559,Indian Overseas Bank,2021-07-01,27.35,IOB IN Equity,INE565A01014,Financials,-2.495544,2.782294,0.987128,1.811401,...,13.0,1,69.0,4,45.0,3,109.0,5,113.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422416,Vodafone Idea Ltd.,2021-07-01,9.05,IDEA IN Equity,INE669E01016,Communication Services,-9.045226,-1.637446,-1.705652,-1.940445,...,217.0,10,216.0,10,215.0,10,213.0,10,193.0,9.0
233567,Kotak Mahindra Bank Ltd.,2021-07-01,1716.10,KMB IN Equity,INE237A01028,Financials,0.600873,-1.672256,-2.246836,-0.549693,...,154.0,8,143.0,7,172.0,8,172.0,8,167.0,8.0
66045,Bandhan Bank Ltd.,2021-07-01,326.30,BANDHAN IN Equity,INE545U01014,Financials,-1.196064,-1.965033,-2.264901,-0.883040,...,178.0,9,195.0,9,209.0,10,209.0,10,200.0,10.0
31378,Amara Raja Energy & Mobility Ltd.,2021-07-01,744.85,ARENM IN Equity,INE885A01032,Industrials,-0.026844,-1.971996,-2.776734,-1.455599,...,207.0,10,206.0,10,197.0,10,194.0,9,33.0,2.0


In [82]:
df_mom.head()

,Company Name,date,Price,BB Symbol,ISIN,Sector,returns,momentum_6,momentum_RA_6,momentum_9,...,momentum_9_rank,momentum__9_decile,momentum_RA_9_rank,momentum_RA_9_decile,momentum_12_rank,momentum__12_decile,momentum_RA_12_rank,momentum_RA_12_decile,MIF_rank,MIF_decile
2205,ABB India Ltd.,2018-01-05,1435.1,ABB IN Equity,INE117A01022,Industrials,0.125584,-0.976181,-1.076045,-0.475453,...,134.0,8,126.0,7,100.0,6,104.0,6,NaN,NaN
2206,ABB India Ltd.,2018-01-08,1437.6,ABB IN Equity,INE117A01022,Industrials,0.174204,-1.027277,-1.169179,-0.444239,...,133.0,7,124.0,7,102.0,6,110.0,6,NaN,NaN
2207,ABB India Ltd.,2018-01-09,1419.3,ABB IN Equity,INE117A01022,Industrials,-1.272955,-1.035835,-1.202220,-0.779455,...,156.0,9,153.0,8,106.0,6,109.0,6,NaN,NaN
2208,ABB India Ltd.,2018-01-10,1415.4,ABB IN Equity,INE117A01022,Industrials,-0.274783,-1.111463,-1.332743,-0.853032,...,160.0,9,160.0,9,105.0,6,106.0,6,NaN,NaN
2209,ABB India Ltd.,2018-01-11,1399.6,ABB IN Equity,INE117A01022,Industrials,-1.116292,-1.128813,-1.388814,-0.997627,...,164.0,9,164.0,9,109.0,6,110.0,6,NaN,NaN


In [3]:
# save the file containing sec and their daily momentum

In [79]:
df_mom.to_parquet('momentum_NSE_200_updated_21stNov.parquet')

In [ ]:
## END